<a href="https://colab.research.google.com/github/RadwaFathi/ImageProcessing-ComputerVision/blob/main/ImageProject1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import needed libraries

In [ ]:
#add imports
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive
import gdown
from moviepy.editor import ImageSequenceClip

  if event.key is 'enter':



# 1. Video Collection

In [ ]:
# Download video from Google Drive
shared_link = "https://drive.google.com/file/d/1FlF50_uqXuBCRx1Mu5FaCvxUwz5EqWEs/view?usp=sharing"
file_id = shared_link.split("/d/")[1].split("/view")[0]
download_url = f"https://drive.google.com/uc?id={file_id}"
video_path = "Computer_Vision_Car_Video.mp4"
gdown.download(download_url, video_path, quiet=False)
print(f"Video downloaded successfully as {video_path}")

Downloading...
From: https://drive.google.com/uc?id=1FlF50_uqXuBCRx1Mu5FaCvxUwz5EqWEs
To: /content/Computer_Vision_Car_Video.mp4
100%|██████████| 3.03M/3.03M [00:00<00:00, 49.6MB/s]

Video downloaded successfully as Computer_Vision_Car_Video.mp4


# 2. Video Preprocessing

In [ ]:
# Create directory for frames
frames_dir = "/content/frames"
os.makedirs(frames_dir, exist_ok=True)

cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
 # Extract at least 20 frames per second

frame_interval = int(fps / 20)
frame_count = 0
# Store some selected frames for further processing
selected_frames = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if frame_count % frame_interval == 0:
        #convert frames to gray scale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        frame_filename = os.path.join(frames_dir, f"frame_{frame_count}.jpg")

        #save frames to a directory
        cv2.imwrite(frame_filename, gray_frame)
        selected_frames.append(gray_frame)

    frame_count += 1

cap.release()
print(f"Extracted {len(selected_frames)} frames.")

Extracted 120 frames.


# 3. Apply Image Processing Techniques

Histogram Analysis:

In [ ]:
# Store histogram visualization frames
histogram_frames = []
histogram_output_dir = "histogram_analysis"
os.makedirs(histogram_output_dir, exist_ok=True)

for i, gray_frame in enumerate(selected_frames):
    # Create histogram visualization
    plt.figure(figsize=(10, 4))
    plt.hist(gray_frame.ravel(), bins=256, range=[0,256], color='gray', alpha=0.7)
    plt.title(f"Histogram of Frame {i}")
    plt.xlabel("Pixel Intensity")
    plt.ylabel("Frequency")

    # Save the histogram plot to an image file
    hist_filename = os.path.join(histogram_output_dir, f"hist_{i:04d}.png")
    plt.savefig(hist_filename)
    plt.close()

    # Read back the saved image and convert to OpenCV format
    hist_img = cv2.imread(hist_filename)
    histogram_frames.append(hist_img)

    # Create side-by-side comparison (original frame + histogram)
    frame_colored = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    frame_resized = cv2.resize(frame_colored, (hist_img.shape[1], hist_img.shape[0]))
    comparison = np.hstack((frame_resized, hist_img))

    # Save the comparison image
    comparison_filename = os.path.join(histogram_output_dir, f"compare_{i:04d}.jpg")
    cv2.imwrite(comparison_filename, comparison)

print(f"Histogram analysis applied to {len(selected_frames)} frames.")
print(f"Histogram images saved in {histogram_output_dir} directory")

Histogram analysis applied to 120 frames.
Histogram images saved in histogram_analysis directory


Histogram Equalization:

In [ ]:
# Store equalized frames
equalized_frames = []
comparison_output_dir = "equalized_comparison"
os.makedirs(comparison_output_dir, exist_ok=True)

for i, gray_frame in enumerate(selected_frames):
    equalized = cv2.equalizeHist(gray_frame)
    equalized_frames.append(equalized)

    # Stack original and equalized side by side for comparison
    comparison = np.hstack((gray_frame, equalized))

    # Save the side-by-side image
    comparison_filename = os.path.join(comparison_output_dir, f"compare_{i:04d}.jpg")
    cv2.imwrite(comparison_filename, comparison)

print(f"Histogram equalization applied to {len(equalized_frames)} frames.")
print(f"Comparison images saved in {comparison_output_dir}")


Histogram equalization applied to 120 frames.
Comparison images saved in equalized_comparison
